In [1]:
import pandas as pd
import numpy as np
import requests
from keras.preprocessing import image
import ast
from PIL import Image
from io import BytesIO
from PIL import ImageEnhance
from sklearn.metrics.pairwise import cosine_similarity
from keras.applications.resnet50 import preprocess_input
from ast import literal_eval

In [70]:
import re
# Loading the data with features
data = pd.read_csv("A2_Data_with_features.csv")
# with open("tfidf_score.pkl", "rb") as f:
#     data2 = pd.read_pickle(f)
data2 = pd.read_csv("tfidf_score.csv")
# data2['Review Text'] = data2['Review Text'].apply(lambda x: ' '.join(x.split('\n')))

def clean_text(text):
    # Remove newline characters
    cleaned_text = text.replace('\n', ' ')
    # Remove multiple consecutive spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()

# Apply the cleaning function to the 'Review Text' column
data2['Review Text'] = data2['Review Text'].apply(clean_text)

data2.to_csv('updated_tfidf_score.csv', index=False)
data2.to_pickle('updated_tfidf_score.pkl')

In [52]:
# Converting 'Feature Extracted' column from string to numpy arrays
data['Feature Extracted'] = data['Feature Extracted'].apply(literal_eval)

In [53]:
# Initialize an empty list to store image features
image_features_matrix = []

# Append all the rows of data['Feature Extracted'] to image_features_matrix
for index, row in data.iterrows():
    image_features_matrix.append(row['Feature Extracted'])

# Convert the list of lists to a numpy array
image_features_matrix = np.array(image_features_matrix)

# Check the shape of the resulting matrix
print(image_features_matrix.shape)


(1640, 2048)


In [54]:
import numpy as np
from numpy.linalg import norm

def get_cosine_similarity(P, Q):
    # If P or Q has zero norm, return 0.0
    if norm(P) == 0 or norm(Q, axis=1).any() == 0:
        return np.zeros(Q.shape[0])  # Return array of zeros
    
    # If both P and Q are single vectors
    if P.ndim == 1 and Q.ndim == 1:
        cosine = np.dot(P, Q) / (norm(P) * norm(Q))
    elif P.ndim == 1 and Q.ndim == 2:
        cosine = np.dot(Q, P) / (norm(P) * norm(Q, axis=1))
    elif P.ndim == 2 and Q.ndim == 2:
        cosine = np.sum(P * Q, axis=1) / (norm(P, axis=1) * norm(Q, axis=1))
    return cosine


In [78]:
store_composite_similarity_scores = []

In [79]:
def find_similar_images(image_url, input_text, top_n=4):
    global store_composite_similarity_scores
    ids = data['ID']
    tfidf_matrix = data2['TF-IDF'].apply(ast.literal_eval)  # Convert strings to arrays
    tfidf_matrix = np.array(tfidf_matrix.tolist())  # Convert Series to 2D array
    print("input text:", input_text)
    text_review = data2['Review Text']
    print(type(tfidf_matrix))
    index = text_review[text_review == input_text].index[0]
    print("See the index:", index)
    tfidf_value = tfidf_matrix[index]
    
    tfidf_value = tfidf_value.reshape(1, -1)
    tfidf_matrix = tfidf_matrix.reshape(len(tfidf_matrix), -1) 
    
    input_image_features = data[data["Image"] == image_url]["Feature Extracted"]
    input_image_features = np.array(input_image_features.apply(ast.literal_eval).tolist())
    input_image_features = input_image_features.reshape(1, -1)
    print("type of input_image_features:", type(input_image_features))
    print("input_image_features: ", input_image_features)
    print(input_image_features.shape)
    print("image_features_matrix:", image_features_matrix.shape)
    print("type of image_features_matrix: ", type(image_features_matrix))
    # Calculate cosine similarity between input image and all other images
    similarity_score = get_cosine_similarity(input_image_features, image_features_matrix)

    similarity_score = similarity_score.flatten()
    
    print(similarity_score)
    print("similarity_score:", similarity_score.shape)
    similarity_dict = {}

    for id_value, sim_score in zip(ids, similarity_score):
        similarity_dict[id_value] = sim_score

    print(similarity_dict)
    
    sorted_similarity = sorted(zip(ids, similarity_score), key=lambda x: x[1], reverse=True)

    top_three = sorted_similarity[:4]

    print(top_three)

    top_three_ids = [pair[0] for pair in top_three]
    cosine_values = [pair[1] for pair in top_three]
    print(top_three_ids)
    # Lists to store image URLs and reviews
    image_urls_list = []
    reviews_list = []

    # Iterate through the top three IDs
    for idx, id_value in enumerate(top_three_ids, 1):
        # Fetch image URLs and review corresponding to the ID
        image_urls_str = data2[data2['ID'] == id_value]['Image'].tolist()[0] 
        image_urls = ast.literal_eval(image_urls_str)   
        review = data2[data2['ID'] == id_value]['Review Text'].tolist()[0]
        print("Review:", review)
        cosine_value = cosine_values[idx - 1]
        output_index = data2[data2['Review Text'] == review].index[0]
        print("output_index:", output_index)
        output_tfidf = tfidf_matrix[output_index]

        output_tfidf = output_tfidf.reshape(1, -1)

        # Append to the lists
        image_urls_list.append(image_urls)
        reviews_list.append(review)

        print("Shape of tfidf_value:", output_tfidf)
        print("Check:", tfidf_value == output_tfidf)
        print("Shape of output_tfidf: ", output_tfidf.shape)
        print("Shape of tfidf_value:", tfidf_value.shape)
        text_similarity_score = (get_cosine_similarity(tfidf_value, output_tfidf))[0]
        composite_similarity_score = (cosine_value + text_similarity_score) / 2
        store_composite_similarity_scores.append(composite_similarity_score)
        if cosine_value > 1.0:
            cosine_value = 1.0

        if text_similarity_score > 1.0:
            text_similarity_score = 1.0
        
        # Print the ID, image URLs, review, and similarity score
        print(f"Top {idx}")
        print(f"ID: {id_value}")
        print(f"Image URLs: {image_urls}")
        print(f"Review: {review}")
        print(f"Cosine Similarity of images: {cosine_value}")
        print(f"Cosine Similarity of text: {text_similarity_score}")
        print(f"Composite Similarity score: {composite_similarity_score}")
        print()


In [72]:
input_image_url = input("Enter the image url: ")
input_review_text = input("Enter the review text: ")

find_similar_images(input_image_url,input_review_text)

input text: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
<class 'numpy.ndarray'>
See the index: 0
type of input_image_features: <class 'numpy.ndarray'>
input_image_features:  [[-0.72990662  0.65725505 -0.85984629 ... -0.39919412  0.61793244
  -0.74109715]]
(1, 2048)
image_features_matrix: (1640, 2048)
type of image_features_matrix:  <class 'numpy.ndarray'>
[1.         0.4620135  0.63672621 ... 0.50107149 0.11890629 0.37691671]
similarity_score: (1640,)
{3452: 1.0, 1205: 0.5368855015258513, 1708: 0.3819374065228087, 2078: 0.6223285984795539, 801: 0.5171227410159202, 126: 0.2908516632388201, 1329: 0.32477613572340003, 325: 0.5757181306944377, 245: 0.10422793638702435, 1714: 0.49090030016577885, 1743: 0.28637289164176266, 3710: 0.3330272575216114, 1664: 0.3327386477328808, 394: 0.4482799348990763, 1819: 0.29741341298721363, 672: 0.354717257

In [80]:
import ast
import numpy as np
import pandas as pd
import pickle

with open("updated_tfidf_score.pkl", "rb") as f:
    df = pd.read_pickle(f)


def find_text_similarity(image_url,input_text):
    global store_composite_similarity_scores
    id = df['ID']
    tfidf_matrix = df['TF-IDF']
    text_review = df['Review Text']

    url_link = data['Image']
    print(type(tfidf_matrix))
    index = url_link[url_link == image_url].index[0]

    input_url_feature = image_features_matrix[index]
    input_url_feature = input_url_feature.reshape(1,-1)

    tfidf_matrix = np.array([np.fromstring(row, dtype=float, sep=',') for row in tfidf_matrix])
    review_index = text_review[text_review == input_text].index
    print(review_index)
    first_occurrance = review_index[0]
    print(first_occurrance)

    if review_index.empty:
        print("Review text not found. It must be outside dataset.")
        tfidf_value = np.zeros_like(tfidf_matrix[0])
    else:
        tfidf_value = tfidf_matrix[first_occurrance]
        # print(tfidf_value)

    tfidf_value = tfidf_value.reshape(1,-1)
    tfidf_matrix = tfidf_matrix.reshape(len(tfidf_matrix),-1)

    similarity = get_cosine_similarity(tfidf_value,tfidf_matrix)

    similarity = similarity.flatten()

    similarity_dict = {}

    for id_value, sim_score in zip(id, similarity):
        similarity_dict[id_value] = sim_score

    print(similarity_dict)

    sorted_similarity = sorted(zip(id, similarity), key=lambda x: x[1], reverse=True)

    top_three = sorted_similarity[:3]

    print(top_three)

    top_three_ids = [pair[0] for pair in top_three]
    cosine_values = [pair[1] for pair in top_three]
    print(top_three_ids)
    # Lists to store image URLs and reviews
    image_urls_list = []
    reviews_list = []

    # Iterating over the top three IDs
    for idx, (id_value) in enumerate(top_three_ids, 1):
        # Fetch image URLs and review corresponding to the ID
        image_urls_str = df[df['ID'] == id_value]['Image'].tolist()[0] 
        image_urls = ast.literal_eval(image_urls_str)   
        review = df[df['ID'] == id_value]['Review Text'].tolist()[0]   
        cosine_value = cosine_values[idx - 1]
        # Append to the lists

        url_mapping = {}

        for url in image_urls:
            features_input_image = data[data["Image"] == url]["Feature Extracted"]
            features_input_image = np.array(features_input_image.values[0])

            index = data[data["Image"] == url].index

            # Checking if match exists
            if not index.empty:
                index = index[0]  # Extract the index value from the pandas Index object
                # Retrieve the corresponding value of "Feature Extracted"
                features_input_image = data.loc[index, "Feature Extracted"]
                print("Index of the row:", index)
                print("Features extracted for the image:", features_input_image)
            else:
                print("No row found with the given URL:", url)

            similarity_score = get_cosine_similarity(input_url_feature,image_features_matrix)

            print(similarity_score)
            similarity_score = similarity_score.flatten()
            # similarity_score = similarity_score[index]
            print(similarity_score)
            print(type(similarity_score))
            print("similarity_score:",similarity_score.shape)
            
            url_mapping[url] = similarity_score

        max_key = max(url_mapping, key=url_mapping.get)
        max_cosine_value = url_mapping[max_key]

        image_urls_list.append(image_urls)
        reviews_list.append(review)

        composite_similarity_score = (cosine_value+max_cosine_value[0])/2
        store_composite_similarity_scores.append(composite_similarity_score)
        
        if cosine_value > 1.0000000000000000:
            cosine_value = 1.0000000000000000

        if max_cosine_value[0] > 1.0000000000000000:
            max_cosine_value[0] = 1.0000000000000000
            
        # Print the ID, image URLs, review, and similarity score
        print(f"Top {idx}")
        print(f"ID: {id_value}")
        print(f"Image URLs: {image_urls}")
        print(f"Review: {review}")
        print(f"Cosine Similarity of text: {cosine_value}")
        print(f"Cosine Similarity of image: {max_cosine_value[0]}")
        print(f"Composite Similarity score: {composite_similarity_score}")
        print()



In [81]:
input_image_url = input("Enter the image url: ")
input_review_text = input("Enter the review text: ")

find_text_similarity(input_image_url,input_review_text)

In [82]:
def retrieve_information(input_image_url,input_review_text):
    print(21*"-")
    print()
    print("USING IMAGE RETRIEVAL")
    find_similar_images(input_image_url,input_review_text)
    print()
    print(21*"-")
    print()
    print("USING TEXT RETRIEVAL")
    find_text_similarity(input_image_url,input_review_text)

In [83]:
input_image_url = input("Enter the image url: ")
input_review_text = input("Enter the review text: ")
retrieve_information(input_image_url,input_review_text)

---------------------

USING IMAGE RETRIEVAL
input text: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
<class 'numpy.ndarray'>
See the index: 0
type of input_image_features: <class 'numpy.ndarray'>
input_image_features:  [[-0.72990662  0.65725505 -0.85984629 ... -0.39919412  0.61793244
  -0.74109715]]
(1, 2048)
image_features_matrix: (1640, 2048)
type of image_features_matrix:  <class 'numpy.ndarray'>
[1.         0.4620135  0.63672621 ... 0.50107149 0.11890629 0.37691671]
similarity_score: (1640,)
{3452: 1.0, 1205: 0.5368855015258513, 1708: 0.3819374065228087, 2078: 0.6223285984795539, 801: 0.5171227410159202, 126: 0.2908516632388201, 1329: 0.32477613572340003, 325: 0.5757181306944377, 245: 0.10422793638702435, 1714: 0.49090030016577885, 1743: 0.28637289164176266, 3710: 0.3330272575216114, 1664: 0.3327386477328808, 394: 0.4482799348990763

In [84]:
print(store_composite_similarity_scores)

[1.0, 0.3788018178493389, 0.3759726738085201, 0.3584988328853608, 1.0, 0.6452534700155337, 0.6075119802672696]


In [87]:
store_composite_similarity_scores = sorted(store_composite_similarity_scores, reverse=True)
print(store_composite_similarity_scores)

[1.0, 1.0, 0.6452534700155337, 0.6075119802672696, 0.3788018178493389, 0.3759726738085201, 0.3584988328853608]
